# 简单项目1：垃圾邮件分类

## 项目描述

使用逻辑回归构建垃圾邮件分类器。这是一个典型的文本分类问题，展示如何将逻辑回归应用于实际场景。

## 学习目标

通过本项目，你将学会：
1. 如何处理文本数据（TF-IDF向量化）
2. 如何使用逻辑回归进行文本分类
3. 如何评估分类模型性能
4. 如何分析特征重要性

## 项目流程

1. 数据加载：加载文本数据
2. 特征提取：使用TF-IDF向量化文本
3. 模型训练：训练逻辑回归模型
4. 模型评估：评估模型性能
5. 特征分析：分析重要特征


## 1. 环境准备


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
np.random.seed(42)
%matplotlib inline

print("环境准备完成！")


## 2. 数据加载

使用20newsgroups数据集的部分类别作为示例。在实际应用中，应该使用真实的垃圾邮件数据集。


In [ ]:
# 加载数据
# 使用20newsgroups数据集的两个类别作为示例
categories = ['alt.atheism', 'soc.religion.christian']

print("加载数据...")
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, 
                                      shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, 
                                     shuffle=True, random_state=42)

X_train = newsgroups_train.data
y_train = newsgroups_train.target
X_test = newsgroups_test.data
y_test = newsgroups_test.target

print(f"训练集大小: {len(X_train)}")
print(f"测试集大小: {len(X_test)}")
print(f"\n类别: {categories}")
print(f"训练集类别分布:")
for i, cat in enumerate(categories):
    print(f"  {cat}: {np.sum(y_train == i)} 个样本")


## 3. 特征提取

使用TF-IDF（Term Frequency-Inverse Document Frequency）将文本转换为数值特征。


In [ ]:
# TF-IDF向量化
# max_features: 最大特征数量（词汇表大小）
# stop_words: 停用词（如'the', 'a'等常见词）
# lowercase: 转换为小写
# strip_accents: 去除重音符号
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english', 
                            lowercase=True, strip_accents='unicode')

print("特征提取...")
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"特征维度: {X_train_tfidf.shape[1]}")
print(f"训练集特征矩阵形状: {X_train_tfidf.shape}")
print(f"测试集特征矩阵形状: {X_test_tfidf.shape}")


## 4. 模型训练

使用逻辑回归训练分类模型。


In [ ]:
# 创建和训练模型
print("训练模型...")
model = LogisticRegression(random_state=42, max_iter=1000, solver='lbfgs')
model.fit(X_train_tfidf, y_train)
print("模型训练完成！")

# 查看模型参数
print(f"\n模型参数:")
print(f"偏置项: {model.intercept_[0]:.4f}")
print(f"权重数量: {len(model.coef_[0])}")
print(f"权重范围: [{model.coef_[0].min():.4f}, {model.coef_[0].max():.4f}]")


## 5. 模型评估

评估模型在测试集上的性能。


In [ ]:
# 预测
y_pred = model.predict(X_test_tfidf)
y_proba = model.predict_proba(X_test_tfidf)[:, 1]

# 评估指标
accuracy = accuracy_score(y_test, y_pred)
print(f"准确率: {accuracy:.4f}")

# 分类报告
print("\n分类报告:")
print(classification_report(y_test, y_pred, target_names=categories))

# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print("\n混淆矩阵:")
print(cm)

# ROC曲线
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
print(f"\nROC AUC: {roc_auc:.4f}")


## 6. 结果可视化

可视化混淆矩阵和ROC曲线。


In [ ]:
# 可视化结果
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. 混淆矩阵
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=categories,
            yticklabels=categories, ax=axes[0])
axes[0].set_title('混淆矩阵')
axes[0].set_ylabel('真实标签')
axes[0].set_xlabel('预测标签')

# 2. ROC曲线
axes[1].plot(fpr, tpr, label=f'ROC曲线 (AUC = {roc_auc:.2f})', linewidth=2)
axes[1].plot([0, 1], [0, 1], 'k--', label='随机分类器')
axes[1].set_xlabel('假正例率 (FPR)')
axes[1].set_ylabel('真正例率 (TPR)')
axes[1].set_title('ROC曲线')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7. 特征重要性分析

分析哪些词对分类最重要。


In [ ]:
# 分析重要特征
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

# 获取最重要的特征
top_n = 20
top_positive = np.argsort(coefficients)[-top_n:][::-1]  # 正系数最大的
top_negative = np.argsort(coefficients)[:top_n]  # 负系数最小的（绝对值最大）

print(f"最重要的 {top_n} 个正特征（倾向于类别1）:")
for idx in top_positive:
    print(f"  {feature_names[idx]}: {coefficients[idx]:.4f}")

print(f"\n最重要的 {top_n} 个负特征（倾向于类别0）:")
for idx in top_negative:
    print(f"  {feature_names[idx]}: {coefficients[idx]:.4f}")

# 可视化特征重要性
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 正特征
top_pos_words = [feature_names[idx] for idx in top_positive]
top_pos_coefs = [coefficients[idx] for idx in top_positive]
axes[0].barh(range(len(top_pos_words)), top_pos_coefs)
axes[0].set_yticks(range(len(top_pos_words)))
axes[0].set_yticklabels(top_pos_words)
axes[0].set_xlabel('系数值')
axes[0].set_title(f'最重要的 {top_n} 个正特征')
axes[0].invert_yaxis()

# 负特征
top_neg_words = [feature_names[idx] for idx in top_negative]
top_neg_coefs = [coefficients[idx] for idx in top_negative]
axes[1].barh(range(len(top_neg_words)), top_neg_coefs, color='salmon')
axes[1].set_yticks(range(len(top_neg_words)))
axes[1].set_yticklabels(top_neg_words)
axes[1].set_xlabel('系数值')
axes[1].set_title(f'最重要的 {top_n} 个负特征')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()


## 8. 总结

### 项目总结

通过本项目，我们：
1. ✅ 学会了如何处理文本数据（TF-IDF向量化）
2. ✅ 使用逻辑回归进行文本分类
3. ✅ 评估了模型性能（准确率、ROC曲线、混淆矩阵）
4. ✅ 分析了特征重要性（哪些词对分类最重要）

### 改进方向

1. **数据质量**：使用真实的垃圾邮件数据集
2. **特征工程**：尝试不同的特征提取方法（如n-gram）
3. **模型优化**：调整正则化参数，尝试不同的C值
4. **特征选择**：使用L1正则化进行特征选择
5. **交叉验证**：使用交叉验证评估模型

### 思考问题

1. 为什么TF-IDF比简单的词频（TF）更好？
2. 如何解释逻辑回归的系数？
3. 如何处理类别不平衡问题？
